# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [1]:
# !apt-get update && apt-get install -y python3-opencv

In [2]:
# !pip install sklearn
# !pip install tqdm
# !pip install torchvision
# !pip install wandb

In [3]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

## Set Arguments & hyperparameters

In [4]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [5]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 30
BATCH_SIZE = 32
LEARNING_RATE = 0.0005
EARLY_STOPPING_PATIENCE = 10

# vggnet, resnet = 224
# basecnn = 128
INPUT_SHAPE = 224

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# DEVICE = torch.device('cpu')
DEVICE

device(type='cuda')

In [6]:
torch.__version__

'1.7.1'

In [7]:
# import random
# import numpy as np
# import os
# import cv2
# import glob
# from PIL import Image
# import PIL.ImageOps    

# #다음 변수를 수정하여 새로 만들 이미지 갯수를 정합니다.
# num_augmented_images = 400

# file_path = 'data/train/'
# file_names = os.listdir(file_path)
# total_origin_image_num = len(file_names)
# augment_cnt = 1

# for i in range(1, num_augmented_images):
#     change_picture_index = random.randrange(1, total_origin_image_num-1)
#     print(change_picture_index)
#     print(file_names[change_picture_index])
#     file_name = file_names[change_picture_index]
    
#     origin_image_path = 'data/train/' + file_name
#     print(origin_image_path)
#     image = Image.open(origin_image_path)
#     random_augment = random.randrange(1,4)
    
#     if(random_augment == 1):
#         #이미지 좌우 반전
#         print("invert")
#         inverted_image = image.transpose(Image.FLIP_LEFT_RIGHT)
#         inverted_image.save(file_path + 'inverted_' + str(augment_cnt) + '.png')
        
#     elif(random_augment == 2):
#         #이미지 기울이기
#         print("rotate")
#         rotated_image = image.rotate(random.randrange(-20, 20))
#         rotated_image.save(file_path + 'rotated_' + str(augment_cnt) + '.png')
        
#     elif(random_augment == 3):
#         #노이즈 추가하기
#         img = cv2.imread(origin_image_path)
#         print("noise")
#         row,col,ch= img.shape
#         mean = 0
#         var = 0.1
#         sigma = var**0.5
#         gauss = np.random.normal(mean,sigma,(row,col,ch))
#         gauss = gauss.reshape(row,col,ch)
#         noisy_array = img + gauss
#         noisy_image = Image.fromarray(np.uint8(noisy_array)).convert('RGB')
#         noisy_image.save(file_path + 'noiseAdded_' + str(augment_cnt) + '.png')
        
#     augment_cnt += 1

## Dataloader

#### Train & Validation Set loader

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']


# Model

## VGGNet 

In [9]:
VGG_types = {
    'VGG11' : [64, 'M', 128, 'M', 256, 256, 'M', 512,512, 'M',512,512,'M'],
    'VGG13' : [64,64, 'M', 128, 128, 'M', 256, 256, 'M', 512,512, 'M', 512,512,'M'],
    'VGG16' : [64,64, 'M', 128, 128, 'M', 256, 256,256, 'M', 512,512,512, 'M',512,512,512,'M'],
    'VGG19' : [64,64, 'M', 128, 128, 'M', 256, 256,256,256, 'M', 512,512,512,512, 'M',512,512,512,512,'M']
}

In [10]:
# define VGGnet class
class VGGnet(nn.Module):
    def __init__(self, model, in_channels=3, num_classes=NUM_CLS, init_weights=True):
        super(VGGnet,self).__init__()
        self.in_channels = in_channels

        # create conv_layers corresponding to VGG type
        self.conv_layers = self.create_conv_laters(VGG_types[model])

        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        
        self.softmax = nn.Softmax(dim=1)

        # weight initialization
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 512 * 7 * 7)
        x = self.fcs(x)
        x = self.softmax(x)
        return x

    # defint weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    # define a function to create conv layer taken the key of VGG_type dict 
    def create_conv_laters(self, architecture):
        layers = []
        in_channels = self.in_channels # 3

        for x in architecture:
            if type(x) == int: # int means conv layer
                out_channels = x

                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                     kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                           nn.BatchNorm2d(x),
                           nn.ReLU()]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
        
        return nn.Sequential(*layers)

# define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# creat VGGnet object
model = VGGnet('VGG11', in_channels=3, num_classes=NUM_CLS, init_weights=True).to(device)
print(model)

cuda
VGGnet(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=(2, 2),

## torchvision.models 

In [11]:
# from torchvision import models

# model = models.resnet18(pretrained=False).to(DEVICE)
# model = models.vgg16(pretrained=False).to(DEVICE)

# model

## base CNN 

In [12]:
# import torch.nn.functional as F

# class custom_CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(custom_CNN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
#         self.pool = nn.MaxPool2d(kernel_size=2)
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        
#         self.fc1 = nn.Linear(in_features=25*29*29, out_features=128)
#         self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
#         x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        
#         x = torch.flatten(x,1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        
#         output = self.softmax(x)
        
#         return output

# summary 

In [13]:
!pip install torchsummary
import torchsummary

torchsummary.summary(model, input_size = (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
         MaxPool2d-4         [-1, 64, 112, 112]               0
            Conv2d-5        [-1, 128, 112, 112]          73,856
       BatchNorm2d-6        [-1, 128, 112, 112]             256
              ReLU-7        [-1, 128, 112, 112]               0
         MaxPool2d-8          [-1, 128, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         295,168
      BatchNorm2d-10          [-1, 256, 56, 56]             512
             ReLU-11          [-1, 256, 56, 56]               0
           Conv2d-12          [-1, 256, 56, 56]         590,080
      BatchNorm2d-13          [-1, 256, 56, 56]             512
             ReLU-14          [-1, 256,

## Utils
### EarlyStopper

In [14]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단


    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [15]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()

            
            pred = self.model(img)
            
            # pred = torch.softmax(pred)
            # pred = torch.round(pred)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            # pred = torch.softmax(pred)
            # pred = torch.round(pred)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [16]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [17]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
# train_dataset2 = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)

# train_dataset = torch.utils.data.ConcatDataset([train_dataset, train_dataset2])

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

#581 - 65

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


#### Load model and other utils

In [18]:
# Load Model
# model = custom_CNN(NUM_CLS).to(DEVICE)


# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn

# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

### epoch 단위 학습 진행

In [19]:
import wandb
# wandb.login(relogin = True)

wandb.init(project = 'test2', name = 'vgg11')

config = wandb.config
config.learning_rate = LEARNING_RATE

for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)
    
    
    wandb.log({"train_loss": trainer.train_mean_loss,
             "train_score": trainer.train_score,
             "val_loss": trainer.val_mean_loss,
             "val_score": trainer.validation_score})

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jw_cho (use `wandb login --relogin` to force relogin)


  0% 0/30 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7909529440932803, Acc: 0.5163511187607573, F1-Macro: 0.5133002429608431


  3% 1/30 [00:50<24:29, 50.69s/it]

Epoch 0, Val loss: 0.9520248770713806, Acc: 0.49230769230769234, F1-Macro: 0.32989690721649484
Epoch 1, Train loss: 0.6777223216162788, Acc: 0.6833046471600689, F1-Macro: 0.6806471656629385


...

 43% 13/30 [11:27<15:07, 53.35s/it]

Epoch 13, Train loss: 0.10664602006889051, Acc: 0.9500860585197934, F1-Macro: 0.9497136342337664


 43% 13/30 [12:17<16:04, 56.72s/it]

Epoch 13, Val loss: 2.1796208173036575, Acc: 0.7384615384615385, F1-Macro: 0.7257383966244726
Early stopping counter 10/10
Early stopped


train_loss,▅▅█▇▆▅▃▃▃▃▂▁▁▁
train_score,▁▄▃▄▄▅▆▆▆▆▇███
val_loss,▂▆█▁▆▁▁▂▁▁▄▁▁▄
val_score,▁▁▅▆▂▄▆▅▅▆▆█▇▇
train_loss,0.10665
train_score,0.95009
val_loss,2.17962
val_score,0.73846


## Inference
### 모델 로드

In [20]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [21]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [22]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [23]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
example_images = []
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:02,  2.28s/it]

tensor([[9.9873e-01, 1.2660e-03],
        [9.9977e-01, 2.3340e-04],
        [9.9596e-01, 4.0385e-03],
        [3.6632e-01, 6.3368e-01],
        [9.9972e-01, 2.7624e-04],
        [5.0853e-02, 9.4915e-01],
        [7.2745e-01, 2.7255e-01],
        [4.5852e-01, 5.4148e-01],
        [8.9140e-01, 1.0860e-01],
        [3.0813e-04, 9.9969e-01],
        [9.9559e-01, 4.4139e-03],
        [9.7065e-01, 2.9345e-02],
        [9.9691e-01, 3.0869e-03],
        [9.9826e-01, 1.7440e-03],
        [9.7067e-01, 2.9333e-02],
        [9.9173e-01, 8.2699e-03],
        [9.9288e-01, 7.1180e-03],
        [9.9999e-01, 1.3135e-05],
        [9.3544e-01, 6.4565e-02],
        [9.9869e-01, 1.3132e-03],
        [9.9962e-01, 3.8238e-04],
        [9.9607e-01, 3.9333e-03],
        [9.4205e-01, 5.7950e-02],
        [9.9866e-01, 1.3445e-03],
        [6.1728e-01, 3.8272e-01],
        [9.9915e-01, 8.5294e-04],
        [9.9879e-01, 1.2052e-03],
        [3.3582e-01, 6.6418e-01],
        [1.5031e-05, 9.9998e-01],
        [1.532

2it [00:04,  2.23s/it]

tensor([[5.7257e-03, 9.9427e-01],
        [9.9667e-01, 3.3319e-03],
        [9.9068e-01, 9.3216e-03],
        [8.5154e-01, 1.4846e-01],
        [3.1522e-01, 6.8478e-01],
        [6.9173e-01, 3.0827e-01],
        [1.7285e-01, 8.2715e-01],
        [8.6064e-01, 1.3936e-01],
        [3.3234e-02, 9.6677e-01],
        [9.5069e-01, 4.9310e-02],
        [9.6109e-01, 3.8910e-02],
        [8.5959e-02, 9.1404e-01],
        [6.2503e-01, 3.7497e-01],
        [1.7761e-01, 8.2239e-01],
        [4.9587e-01, 5.0413e-01],
        [9.9989e-01, 1.0852e-04],
        [9.3796e-01, 6.2036e-02],
        [9.9942e-01, 5.7844e-04],
        [4.0114e-01, 5.9886e-01],
        [5.2926e-03, 9.9471e-01],
        [8.8043e-01, 1.1957e-01],
        [9.9209e-01, 7.9126e-03],
        [4.1224e-01, 5.8776e-01],
        [9.2052e-01, 7.9480e-02],
        [9.6154e-01, 3.8458e-02],
        [1.6352e-01, 8.3648e-01],
        [9.9972e-01, 2.8482e-04],
        [9.3031e-01, 6.9694e-02],
        [2.2678e-03, 9.9773e-01],
        [9.991

4it [00:06,  1.68s/it]

tensor([[8.2033e-02, 9.1797e-01],
        [1.5390e-03, 9.9846e-01],
        [5.0252e-01, 4.9748e-01],
        [9.9343e-01, 6.5723e-03],
        [3.6131e-01, 6.3869e-01],
        [9.8119e-01, 1.8815e-02],
        [9.9897e-01, 1.0309e-03],
        [9.9858e-01, 1.4247e-03],
        [9.9519e-01, 4.8136e-03],
        [9.9888e-01, 1.1216e-03],
        [1.0708e-01, 8.9292e-01],
        [9.9983e-01, 1.6813e-04],
        [9.9984e-01, 1.5988e-04],
        [9.9999e-01, 6.7867e-06],
        [9.1940e-01, 8.0596e-02],
        [1.6587e-01, 8.3413e-01],
        [3.8572e-01, 6.1428e-01],
        [9.7901e-01, 2.0991e-02],
        [9.9004e-01, 9.9645e-03],
        [1.9205e-01, 8.0795e-01],
        [4.3341e-01, 5.6659e-01],
        [6.4614e-01, 3.5386e-01],
        [9.8880e-01, 1.1200e-02],
        [4.4713e-01, 5.5287e-01],
        [9.9541e-01, 4.5938e-03],
        [7.0580e-01, 2.9420e-01],
        [9.9934e-01, 6.5843e-04],
        [9.9758e-01, 2.4212e-03],
        [2.5312e-02, 9.7469e-01],
        [1.062

### 결과 저장

In [24]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('0212_vgg11.csv', index=False)

In [25]:
# df_pred = pd.read_csv('0212_vgg11.csv')
# df_pred.head()

,file_name,COVID
0,0.png,0
1,1.png,0
2,2.png,0
3,3.png,1
4,4.png,0


In [ ]:
# df_pred = pd.read_csv('0212_vgg16.csv')
# df_pred.head()

In [25]:
# df_pred = pd.read_csv('0212_vgg13.csv')
# df_pred.head()

,file_name,COVID
0,0.png,0
1,1.png,0
2,2.png,0
3,3.png,1
4,4.png,0
